In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime


In [2]:
driver = webdriver.Chrome()
driver.get("https://www.indycar.com/Schedule")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('li', {'class': 'schedule-list__item'})

driver.quit()

                                        

In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = []
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year



month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }





In [5]:
circuit_naming = {}
link_names = []
for i in schedule_details:
    link_temp = i.find('a', class_='panel-trigger schedule-list__title')
    link = link_temp['href'].split('/')[-1]
    link_names.append(link)
    
circ_names = ['St Petersburg Street Circuit' '\n' 'St. Petersburg, FL.',
              'Thermal Club Circuit' '\n' 'Thermal, CA.',
              'Long Beach Street Circuit' '\n' 'Long Beach, CA.',
              'Barber Motorsports Park' '\n' 'Birmingham, AL',
              'Indianapolis Motor Speedway Road Course' '\n' 'Indianapolis, IN.',
              'Indianapolis Motor Speedway Oval' '\n' 'Indianapolis, IN.',
              'Downtown Detroit Street Circuit' '\n' 'Detroit, MI',
              'Road America' '\n' 'Elkhart Lake, WI.',
              'Laguna Seca Raceway' '\n' 'Monterey, CA',
              'Mid-Ohio Sports Car Course' '\n' 'Lexington, OH',
              'Iowa Speedway' '\n' 'Newton, IA',
              'Iowa Speedway' '\n' 'Newton, IA',
              'Exhibition Place Street Circuit' '\n' 'Toronto, Ontario, Canada',
              'Gateway Motorsports Park' '\n' 'Madison, IL.',
              'Portland International Raceway' '\n' 'Portland, OR.',
              'Milwaukee Mile' '\n' 'West Allis, WI',
              'Milwaukee Mile' '\n' 'West Allis, WI',
              'Nashville Superspeedway' '\n' 'Lebanon, TN'
             ]

circuit_naming = dict(zip(link_names, circ_names))

In [6]:
##Testing##
# link_prefix = 'https://www.indycar.com'

# # schedule_details[1].find('a', class_='panel-trigger schedule-list__title')
# link_temp = schedule_details[1].find('a', class_='panel-trigger schedule-list__title')
# link = link_temp['href']
# driver = webdriver.Chrome()
# driver.get(link_prefix + link)
# event_soup = BeautifulSoup(driver.page_source, 'html.parser')
# desc = event_soup.find('h2', {'style':''}).text.strip()
# desc

# # <div class="race-list__date text">Sunday,   Sep 15 </div>

In [7]:
link_prefix = 'https://www.indycar.com'
round_counter = 1

for i in schedule_details:
    link_temp = i.find('a', class_='panel-trigger schedule-list__title')
    link = link_temp['href']
    driver = webdriver.Chrome()
    driver.get(link_prefix + link)
    event_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    desc = event_soup.find('h2', {'style':''}).text.strip()
    dist = event_soup.find('h2', {'class':'race-details'}).text.split('|')
    lap_count = dist[0].strip().replace('\n', '').replace('LAPS', 'Laps').split()
    lap_count = ' '.join(lap_count)  # Join the split elements to remove excess space
    distance = dist[1].strip().replace('\n', '')
    formatted_output = f"{lap_count}, {distance}"                  
    descs.append(desc + '\n' + formatted_output)
    
    temp_circuit = link_temp['href'].split('/')[-1]
    converted_circuit = circuit_naming[temp_circuit]
    circuits.append(converted_circuit)
    
    if 'Thermal' in converted_circuit:
        temp_title = 'IndyCar Exhibition Race'
    else:
        temp_title = 'IndyCar Round ' + '0' + str(round_counter)
        round_counter += 1
    titles.append(temp_title)
    
    temp_date = event_soup.find('div', class_='race-list__date text').text.split(' ')
    temp_month = temp_date[-3]
    conv_month = month_mapping[temp_month]
    temp_day = temp_date[-2]
    months.append(conv_month)
    days.append(int(temp_day))
    
    temp_time = event_soup.find('div', class_='race-list__time text').text.split('-')
    if 'TBD' in temp_time:
        start_hours.append(12)
        start_minutes.append(0)
        end_hours.append(15)
        end_minutes.append(0)
        
    else:
        start_h, start_m = temp_time[0].split(':')[0], temp_time[0].split(':')[-1].strip(' PM').strip(' A')
        if 'PM' in temp_time[0]:
            if start_h == '12':
                start_hours.append(int(start_h))
            else:  
                start_h = int(start_h) + 12
                start_hours.append(start_h)
        else:
            start_hours.append(int(start_h))
        if start_m == '00':
            start_minutes.append(0)
        else:
            start_minutes.append(int(start_m))

        end_h, end_m = temp_time[1].split(':')[0], temp_time[0].split(':')[-1].strip(' PM').strip(' A')
        if 'PM' in temp_time[1]:
            end_h = int(end_h) + 12
            end_hours.append(end_h)
        else:
            end_hours.append(int(end_h))
        if end_m == '00':
            end_minutes.append(0)
        else:
            end_minutes.append(int(end_m))
     

In [8]:
indy_pq_month = 5
pq_circ  ='Indianapolis Motor Speedway Oval' '\n' 'Indianapolis, IN.'

pq_days = [14, 15, 16, 17, 18, 18, 19, 19, 19, 19, 19, 20, 24, 24]
pq_s_h = [9, 12, 12, 12, 8, 11, 11, 12, 14, 16, 17, 13, 11, 14]
pq_s_m = [0, 0, 0, 0, 30, 0, 30, 30, 0, 0, 15, 0, 0, 30]
pq_e_h = [18, 18, 18, 18, 9, 17, 12, 13, 15, 17, 17, 15, 13, 16]
pq_e_m = [0, 0, 0, 0, 30, 50, 30, 30, 0, 0, 45, 0, 0, 0]
tits = ['Indy 500 Practice', 'Indy 500 Practice', 'Indy 500 Practice', 'Indy 500 Fast Friday', 'Indy 500 Open Practice',
        'Indy 500 Qualifying', 'Indy 500 Top 12 Practice', 'Indy 500 LCQ Practice', 'Indy 500 Top 12 Qualifying',
        'Indy 500 Last Chance Qualifying', 'Indy 500 Fast Six Qualifying', 'Indy 500 Practice',
        'Indy 500 Carb Day Final Practice', 'Indy 500 Pit Stop Challenge']





In [9]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [10]:
IndyCar = Calendar()
IndyCar.add('prodid', '-//My calendar product//example.com//')
IndyCar.add('version', '2.0')
uid = 110

In [11]:
 
for title, circuit, day, month, desc, s_h, s_m, e_h, e_m in zip(titles, circuits, days, months, descs, start_hours, start_minutes, end_hours, end_minutes):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, s_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(year, month, day, e_h, e_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    IndyCar.add_component(ievent)

In [12]:
for tit, day, s_h, s_m, e_h, e_m in zip(tits, pq_days, pq_s_h, pq_s_m, pq_e_h, pq_e_m):
    ievent = Event()
    ievent.add('summary', tit) #Title of the event
    ievent.add('description', tit)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, indy_pq_month, day, s_h, s_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(year, indy_pq_month, day, e_h, e_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', pq_circ)
    ievent.add('uid', uid)
    uid += 1
    IndyCar.add_component(ievent)

In [13]:
IndyCar

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'IndyCar Round 01''), 'DESCRIPTION': vText('b'Firestone Grand Prix of St. Petersburg\\n100 Laps\\, 180 Miles''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001A8E29DC850>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001A8E29DCF10>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001A8E29DCFA0>, 'LOCATION': vText('b'St Petersburg Street Circuit\\nSt. Petersburg\\, FL.''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'IndyCar Exhibition Race''), 'DESCRIPTION': vText('b'$1 Million Challenge\\nLaps\\, ''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001A8E29DCD90>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001A8E29DCD00>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001A8E29DCDC0>, 'LOCATION': vText('b'Thermal Club Circuit\\nThermal\\, CA.''), 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'IndyCar Round 02''

In [14]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'IndyCar.ics'), 'wb')
f.write(IndyCar.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
